**IMAGE CLASSIFICATION USING CNN**



Importing libraries

In [ ]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import skimage.io as io
from torchvision import datasets
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise
from skimage.filters import gaussian
from torchvision import models
from torchsummary import summary
import pandas as pd

Using the data uploaded on drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Defining the neural network

In [ ]:
# Neural Network model
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # first convolution layer with batch normalization
        self.conv1 = nn.Conv2d(3,6,5)
        self.conv1_bn = nn.BatchNorm2d(6)
        # second convolution layer with batch normalization
        self.conv2 = nn.Conv2d(6,12,5)
        self.conv2_bn = nn.BatchNorm2d(12)
        # third convolution layer with batch normalization
        self.conv3 = nn.Conv2d(12,32,3)
        self.conv3_bn = nn.BatchNorm2d(32)
        # fourth convolution layer with batch normalization
        self.conv4 = nn.Conv2d(32,128,3)
        self.conv4_bn = nn.BatchNorm2d(128)
        # fifth convolution layer with batch normalization
        self.conv5 = nn.Conv2d(128,256,3)
        self.conv5_bn = nn.BatchNorm2d(256)
        # first fully connected layer with batch normalization
        self.fcn1 = nn.Linear(256*5*5,4000)
        self.fcn1_bn = nn.BatchNorm1d(4000)
        # second fully connected layer with batch normalization
        self.fcn2 = nn.Linear(4000,1000)
        self.fcn2_bn = nn.BatchNorm1d(1000)
        # third fully connected layer with batch normalization
        self.fcn3 = nn.Linear(1000,500)
        self.fcn3_bn = nn.BatchNorm1d(500)
        # final output
        self.out = nn.Linear(500,10)
        # dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        # first convolution layer with max pooling and relu activation
        x = self.conv1(x)
        x = self.conv1_bn(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # second convolution layer with max pooling and relu activation
        x = self.conv2(x)
        x = self.conv2_bn(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # third convolution layer with max pooling and relu activation
        x = self.conv3(x)
        x = self.conv3_bn(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # fourth convolution layer with max pooling and relu activation
        x = self.conv4(x)
        x = self.conv4_bn(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # fifth convolution layer with max pooling and relu activation
        x = self.conv5(x)
        x = self.conv5_bn(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # reshaping for fully connected layer
        x = x.reshape(-1, 256*5*5)
        # first fully connected layer with max pooling, dropout and relu activation
        x = self.fcn1(x)
        x = self.fcn1_bn(x)
        x = F.relu(x)
        # second fully connected layer with max pooling, dropout and relu activation
        x = self.dropout(x)
        x = self.fcn2(x)
        x = self.fcn2_bn(x)
        x = F.relu(x)
        # third fully connected layer with max pooling, dropout and relu activation
        x = self.dropout(x)
        x = self.fcn3(x)
        x = self.fcn3_bn(x)
        x = F.relu(x)
        # output
        x = self.out(x)
        # applying softmax
        x = F.softmax(x,dim=1)
        return x

In [ ]:
# creating a Network
network = Network().cuda()

Data Augmentation

In [ ]:
# Augmenting images and saving them in same folder
!pip install Augmentor
import Augmentor
# Augmentation for railway folder
p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/railway',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/railway')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate 
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)
p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/swimming_pool',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/swimming_pool')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate 
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/tennis_court',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/tennis_court')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/bridge',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/bridge')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/oil_well',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/oil_well')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/crosswalk',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/crosswalk')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/overpass',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/overpass')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/basketball_court',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/basketball_court')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/runway',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/runway')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

p = Augmentor.Pipeline('/content/drive/My Drive/GNR638/TrainSet/train/golf_course',output_directory='/content/drive/My Drive/GNR638/TrainSet/train/golf_course')
# flip left/right
p.flip_left_right(probability=0.8)
# flip top/bottom
p.flip_top_bottom(probability=0.8)
# rotate
p.rotate_random_90(probability=0.8)
# shear
p.shear(probability=0.8,max_shear_left=20,max_shear_right=20)
# zoom
p.zoom(probability=0.8,min_factor=1.1, max_factor=1.8)
# skew
p.skew(probability=0.8)
p.sample(500)

# Now we will have 500 + 10*500 = 5500 images

Training the given data

In [ ]:
# training dataset
train_set = datasets.ImageFolder(root='/content/drive/My Drive/GNR638/TrainSet/train', transform=transforms.Compose([transforms.ToTensor()]), target_transform=None, is_valid_file=None)
# using batch size = 10
Training_dataset = torch.utils.data.DataLoader(train_set, batch_size = 10, shuffle = True)

In [ ]:
# using stochastic gradient descent as optimizer and cross entroply loss as loss function
optimizer = optim.SGD(network.parameters(), lr=0.05,momentum=0.6)
loss_func = nn.CrossEntropyLoss()
# test set
test_set = datasets.ImageFolder('/content/drive/My Drive/GNR638/TestSet', transform=transforms.Compose([transforms.ToTensor()]), target_transform=None, is_valid_file=None)
final_test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False)
maxm = 0
# test labels
label_test_set = torch.tensor(np.array([4,9,2,4,6,0,9,3,1,1,6,0,4,5,7,8,0,4,7,9,4,1,5,3,2,5,6,6,4,2,8,0,2,7,1,3,2,1,5,8,9,9,0,3,6,8,9,4,0,8,7,7,0,5,0,3,9,2,6,1,9,5,8,1,6,3,7,8,3,9,4,5,8,1,7,2,6,0,2,8,1,3,1,7,5,9,7,2,4,2,5,3,5,8,6,6,4,3,0,7]))
# epochs = 50
for epoch in range(50):
    # initializing train accuracy and total running loss
    total_loss = 0
    train_acc = 0
    for batch in Training_dataset:
        images, labels = batch
        optimizer.zero_grad()
        preds = network(images.cuda())
        loss = loss_func(preds.cuda(), labels.cuda())
        loss.backward()
        optimizer.step()
        total_loss = total_loss + loss.item()
        train_acc += preds.cuda().argmax(dim=1).eq(labels.cuda()).sum().item()
    result = torch.tensor([])
    for batch1 in final_test_loader:
        image1,label1 = batch1
        prediction = network(image1.cuda())
        result = prediction.argmax(dim=1)
    if torch.eq(label_test_set.cuda(), result).sum().item()>=maxm:
        maxm = torch.eq(label_test_set.cuda(), result).sum().item()
        # saving the model
        torch.save(network, 'model.pkl')
    print(epoch)
    print(train_acc*100/5500)
    print(total_loss)    
    # test accuracy
    print(torch.eq(label_test_set.cuda(), result).sum().item(),'%')
    print('---------------------------------------------------')    

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0
48.763636363636365
1085.2134841680527
42 %
---------------------------------------------------
1
62.30909090909091
1012.2746584415436
48 %
---------------------------------------------------
2
66.69090909090909
986.998117685318
66 %
---------------------------------------------------
3
70.74545454545455
964.4654704332352
65 %
---------------------------------------------------
4
71.03636363636363
962.0825779438019
68 %
---------------------------------------------------
5
74.69090909090909
943.6329208612442
72 %
---------------------------------------------------
6
78.49090909090908
924.651504278183
75 %
---------------------------------------------------
7
78.4
922.4520336389542
69 %
---------------------------------------------------
8
80.78181818181818
909.8121120929718
80 %
---------------------------------------------------
9
80.65454545454546
909.7443891763687
72 %
---------------------------------------------------
10
82.87272727272727
898.2116483449936
73 %
------------------

Testing on the test set given

In [ ]:
# loading the saved model as network
network = torch.load('model.pkl')
test_set = datasets.ImageFolder('/content/drive/My Drive/GNR638/TestSet', transform=transforms.Compose([transforms.ToTensor()]), target_transform=None, is_valid_file=None)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False)
labels = torch.tensor(np.array([4,9,2,4,6,0,9,3,1,1,6,0,4,5,7,8,0,4,7,9,4,1,5,3,2,5,6,6,4,2,8,0,2,7,1,3,2,1,5,8,9,9,0,3,6,8,9,4,0,8,7,7,0,5,0,3,9,2,6,1,9,5,8,1,6,3,7,8,3,9,4,5,8,1,7,2,6,0,2,8,1,3,1,7,5,9,7,2,4,2,5,3,5,8,6,6,4,3,0,7]))
result = torch.tensor([])
for batch1 in test_loader:
    image1,label1 = batch1
    prediction = network(image1.cuda())
    result = prediction.argmax(dim=1)
result = result + 1
a = {'ImageID': range(101, 201), 'LabelID': result.tolist()}
b = pd.DataFrame(a)
# printing test accuracy
print(result.tolist())
# saving the csv file
b.to_csv('/content/drive/My Drive/GNR638/19D070014.csv', index = False)

90 %
[5, 10, 3, 5, 7, 1, 10, 4, 2, 2, 1, 1, 5, 6, 8, 9, 6, 5, 8, 8, 5, 6, 6, 4, 3, 6, 7, 7, 5, 3, 9, 3, 3, 8, 6, 4, 3, 2, 6, 9, 3, 10, 1, 4, 7, 9, 10, 5, 1, 9, 8, 8, 1, 6, 1, 4, 10, 3, 7, 2, 10, 6, 9, 2, 7, 4, 8, 9, 4, 10, 5, 6, 9, 2, 8, 3, 10, 8, 3, 9, 2, 4, 2, 8, 6, 10, 8, 3, 5, 3, 6, 4, 6, 9, 7, 7, 5, 4, 8, 8]
